In [919]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.impute import SimpleImputer
from datetime import datetime

In [920]:
df=pd.read_csv("Marvel_Comics.csv")
df.head()

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price
0,A Year of Marvels: April Infinite Comic (2016),(2016),A Year of Marvels: April Infinite Comic (2016) #1,"April 01, 2016",The Infinite Comic that will have everyone tal...,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,Free
1,A Year of Marvels: August Infinite Comic (2016),(2016),A Year of Marvels: August Infinite Comic (2016...,"August 10, 2016","It’s August, and Nick Fury is just in time to ...",Jamal Campbell,"Chris Sims, Chad Bowers",NaN,Marvel Universe,Infinite Comic,NaN,Free
2,A Year of Marvels: February Infinite Comic (2016),(2016),A Year of Marvels: February Infinite Comic (20...,"February 10, 2016",Join us in a brand new Marvel comics adventure...,"Danilo S. Beyruth, M Mast",Ryan North,NaN,Marvel Universe,Infinite Comic,Rated T+,Free
3,A Year of Marvels: July Infinite Comic (2016),(2016),A Year of Marvels: July Infinite Comic (2016) #1,"June 29, 2016",Celebrating the Fourth of July is complicated ...,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free
4,A Year of Marvels: June Infinite Comic (2016),(2016),A Year of Marvels: June Infinite Comic (2016) #1,"June 15, 2016",Sam Alexander’s finding it hard to cope with t...,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free


In [921]:
df.shape

(34992, 12)

In [922]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34992 entries, 0 to 34991
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   comic_name         34992 non-null  object
 1   active_years       34992 non-null  object
 2   issue_title        34466 non-null  object
 3   publish_date       34466 non-null  object
 4   issue_description  30395 non-null  object
 5   penciler           25482 non-null  object
 6   writer             27595 non-null  object
 7   cover_artist       12255 non-null  object
 8   Imprint            11684 non-null  object
 9   Format             32894 non-null  object
 10  Rating             12619 non-null  object
 11  Price              32894 non-null  object
dtypes: object(12)
memory usage: 3.2+ MB


In [923]:
df.describe()

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price
count,34992,34992,34466,34466,30395,25482,27595,12255,11684,32894,12619,32894
unique,4935,412,33757,3317,29331,3915,3082,1010,39,11,36,53
top,Uncanny X-Men (1981 - 2011),(1968 - 1996),X-Men: The Complete Age of Apocalypse Epic Boo...,"November 30, -0001",Please note that these digital editions collec...,Sal Buscema,Brian Michael Bendis,Gil Kane,Marvel Universe,Comic,Rated T+,Free
freq,588,718,10,543,37,394,859,217,7786,31899,3204,15136


## FASE 1 Diseño de la base de datos 

In [924]:
client = MongoClient("mongodb://localhost:27017")
db = client['marvel']

collections = ["raw_marvel", "curated_marvel", "analytics_marvel"]

for coll in collections:
    if coll not in db.list_collection_names():
        db.create_collection(coll)
        print(f"Colección '{coll}' creada")
    else:
        print(f"Colección '{coll}' ya existe")



Colección 'raw_marvel' ya existe
Colección 'curated_marvel' ya existe
Colección 'analytics_marvel' ya existe


## FASE 2 Carga de datos en RAW

In [925]:
df_raw=df.copy()

In [926]:
df_raw.drop(columns=["issue_description","issue_title"],inplace=True)

In [927]:
#NORMALIZACIÓN DE NOMBRES DE COLUMNAS
cols=df_raw.columns.astype(str).str.title()
print(cols)

Index(['Comic_Name', 'Active_Years', 'Publish_Date', 'Penciler', 'Writer',
       'Cover_Artist', 'Imprint', 'Format', 'Rating', 'Price'],
      dtype='object')


In [928]:
#EXTRAEMOS LOS AÑOS DE LOS PARÉNTESIS Y LOS CONVERTIMOS A ENTEROS
df_raw["active_years"] = df_raw["active_years"].str.extract(r"\((\d{4})")[0].astype("int64")

In [929]:
#CAMBIAMOS LOS PRECIOS A FLOAT
df_raw["Price"] = df_raw["Price"].str.strip().replace("Free", "0")  # Free → 0
df_raw["Price"] = df_raw["Price"].str.replace(r"[\$,]", "", regex=True)  # quitar $ y comas
df_raw["Price"] = df_raw["Price"].astype("float64")

In [930]:
#CAMBIAMOS EL FORMATO DE FECHA
df_raw["publish_date"] = pd.to_datetime(df_raw["publish_date"], errors="coerce")
datetime_cols = df_raw.select_dtypes(include=["datetime64[ns]"]).columns
df_raw[datetime_cols] = df_raw[datetime_cols].astype("object")

#IMPUTAMOS
imputer = SimpleImputer(strategy="most_frequent")
df_raw[datetime_cols] = imputer.fit_transform(df_raw[datetime_cols])
df_raw[datetime_cols] = df_raw[datetime_cols].apply(pd.to_datetime, errors="coerce")
print(df_raw[datetime_cols].isna().sum())


publish_date    0
dtype: int64


In [931]:
df_raw.dtypes

comic_name              object
active_years             int64
publish_date    datetime64[ns]
penciler                object
writer                  object
cover_artist            object
Imprint                 object
Format                  object
Rating                  object
Price                  float64
dtype: object

In [932]:
#CAMBIAMOS LOS NAN DE STRING A NULL
obj_cols = df_raw.select_dtypes(include="O").columns

df_raw[obj_cols] = df_raw[obj_cols].where(df_raw[obj_cols].notna(), None)

In [933]:
df_raw.head()

,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price
0,A Year of Marvels: April Infinite Comic (2016),2016,2016-04-01,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,0.0
1,A Year of Marvels: August Infinite Comic (2016),2016,2016-08-10,Jamal Campbell,"Chris Sims, Chad Bowers",None,Marvel Universe,Infinite Comic,None,0.0
2,A Year of Marvels: February Infinite Comic (2016),2016,2016-02-10,"Danilo S. Beyruth, M Mast",Ryan North,None,Marvel Universe,Infinite Comic,Rated T+,0.0
3,A Year of Marvels: July Infinite Comic (2016),2016,2016-06-29,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,None,0.0
4,A Year of Marvels: June Infinite Comic (2016),2016,2016-06-15,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,None,0.0


In [934]:
print(df_raw.isna().sum())

comic_name          0
active_years        0
publish_date        0
penciler         9510
writer           7397
cover_artist    22737
Imprint         23308
Format           2098
Rating          22373
Price            2098
dtype: int64


In [935]:
print(df_raw.isna().sum()/len(df_raw))

comic_name      0.000000
active_years    0.000000
publish_date    0.000000
penciler        0.271776
writer          0.211391
cover_artist    0.649777
Imprint         0.666095
Format          0.059957
Rating          0.639375
Price           0.059957
dtype: float64


In [936]:
#VAMOS A COMPROBAR QUE CATEGORIAS DE RATING HAY
unique_values = df_raw["Rating"].unique()
print(unique_values)



[' Rated T+' None ' Rated T' ' ALL AGES' ' A' ' Parental Advisory'
 ' Marvel Psr' ' No Rating' ' MARVEL PSR' ' T' ' RATED T' ' Max'
 ' RATED T+' ' RATED A' ' All Ages' ' T+' ' Rated a' ' Rated A'
 ' Parental Advisory/Explicit Content' ' PARENTAL SUPERVISION'
 ' PARENTAL ADVISORY' ' Mature' ' MARVEL PSR+' ' EXPLICIT CONTENT'
 ' PARENTAL ADVISORYSLC' ' Parental AdvisorySLC' ' Parental Advisoryslc'
 ' Explicit Content' ' PARENTAL ADVISORY/EXPLICIT CONTENT' ' NO RATING'
 ' NOT IN ORACLE' ' Parental Guidance' ' Ages 10 & Up' ' Not in Oracle'
 ' MAX' ' Marvel Psr+' ' Ages 9+']


In [937]:
# UNIFICAMOS LAS CATEGORÍAS DE RATING (Manteniendo nulos)
df_raw["Rating_clean"] = df_raw["Rating"].str.lower().str.strip()

rating_map = {
    "t+": "T+",
    "rated t+": "T+",
    "rated t": "T",
    "t": "T",
    "all ages": "All Ages",
    "a": "A",
    "rated a": "A",
    "parental advisory": "Parental Advisory",
    "parental advisory/explicit content": "Parental Advisory",
    "parental advisoryslc": "Parental Advisory",
    "explicit content": "Explicit Content",
    "marvel psr": "Marvel PSR",
    "marvel psr+": "Marvel PSR+",
    "max": "Max",
    "mature": "Mature",
    "parental guidance": "Parental Advisory",
    "parental supervision": "Parental Advisory",
    "ages 10 & up": "All Ages",
    "ages 9+": "All Ages"
}

df_raw["Rating"] = df_raw["Rating_clean"].replace(rating_map)

df_raw.drop(columns=["Rating_clean"], inplace=True)

print(f"Nulos en Rating: {df_raw['Rating'].isna().sum()}")
print(df_raw["Rating"].value_counts(dropna=False))

Nulos en Rating: 22373
Rating
None                 22373
T+                    4410
T                     3057
Parental Advisory     1613
All Ages              1199
Marvel PSR            1060
A                      903
Explicit Content       149
no rating              102
Mature                  44
Max                     34
not in oracle           27
Marvel PSR+             21
Name: count, dtype: int64


In [938]:
#DUPLICADOS
num_duplicates = df_raw.duplicated().sum()
print(f"Hay {num_duplicates} registros duplicados")


Hay 771 registros duplicados


In [939]:
#ELIMINAMOS LOS DUPLICADOS
df_raw = df_raw.drop_duplicates()



In [940]:
#AÑADIMOS UN ID
df_raw["id"] = range(1, len(df_raw) + 1)


In [941]:
coleccion = db["raw_marvel"] 

In [942]:
coleccion.delete_many({})
print("Colección 'raw_marvel' limpia")

Colección 'raw_marvel' limpia


In [943]:
#INSERCIÓN EN LA BASE DE DATOS
data_to_insert = df_raw.to_dict(orient='records')

result = coleccion.insert_many(data_to_insert)

print(f"Se insertaron {len(result.inserted_ids)} documentos en 'raw_marvel'")


Se insertaron 34221 documentos en 'raw_marvel'


## FASE 3 CRUD completo sobre RAW

In [944]:
#CREAMOS UN COMIC
nuevo_comic = {
    "comic_name": "Avengers: Future Fight (2026)",
    "active_years": 2026,
    "publish_date": datetime(2026, 3, 1),
    "penciler": "Emily White",
    "writer": "Mark Black",
    "cover_artist": "Chris Sanders",
    "Imprint": "Marvel Universe",
    "Format": "Graphic Novel",
    "Rating": "T+",
    "Price": 5.99,
    "id": 34223
}


### INSERCIÓN

In [945]:
#INSERTAMOS UN COMIC
result_create = coleccion.insert_one(nuevo_comic)
print(f"Documento insertado con _id: {result_create.inserted_id}")

Documento insertado con _id: 699855c16119c5fcdee7d8f7


### FIND

In [946]:
#ENCONTRAMOS CON FILTRO DE CAMPO CATEGÓRICO
all_ages_comics = pd.DataFrame(coleccion.find({"Rating": "All Ages"}))
print("Dataframe de comics para todas las edades:")
all_ages_comics.head()

Dataframe de comics para todas las edades:


,_id,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id
0,699855c06119c5fcdee75379,A-Next (1998 - 1999),1998,2006-08-09,None,None,None,MARVEL UNIVERSE,Digest,All Ages,7.99,48
1,699855c06119c5fcdee756aa,Amazing Fantasy Facsimile Edition (2019),2019,2019-10-09,None,None,None,Marvel Universe,Comic,All Ages,3.99,865
2,699855c06119c5fcdee757f2,Amazing Spider-Man Annual (1964 - 2018),1964,2005-04-01,None,None,None,MARVEL UNIVERSE,Trade Paperback,All Ages,0.00,1193
3,699855c06119c5fcdee757f4,Amazing Spider-Man Annual (1964 - 2018),1964,2004-01-01,None,None,None,MARVEL UNIVERSE,Hardcover,All Ages,0.00,1195
4,699855c06119c5fcdee75a24,Avengers (1963 - 1996),1963,2005-04-01,None,None,None,MARVEL UNIVERSE,Trade Paperback,All Ages,0.00,1755


In [947]:
#FILTRO DE COMPARACIÓN NUMÉRICA DE PRECIO
comics_con_precio = pd.DataFrame(coleccion.find({"Price": {"$gt": 5.00}}))
print('Dataframe con los comics con precio mayor a 5 dólares:')
comics_con_precio.head()

Dataframe con los comics con precio mayor a 5 dólares:


,_id,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id
0,699855c06119c5fcdee75379,A-Next (1998 - 1999),1998,2006-08-09,None,None,None,MARVEL UNIVERSE,Digest,All Ages,7.99,48
1,699855c06119c5fcdee7538a,Absolute Carnage (2019),2019,2019-08-07,Ryan Stegman,Donny Cates,Ryan Stegman,None,Comic,T+,7.99,65
2,699855c06119c5fcdee754ed,All-New Marvel Now! Point One (2014),2014,2014-01-08,Steve Mcniven,"G. Willow Wilson, Al Ewing, James Robinson, Na...",None,Marvel Universe,Comic,None,5.99,420
3,699855c06119c5fcdee75584,All-New X-Men Annual (2014),2014,2014-12-24,None,None,None,None,Comic,None,24.99,571
4,699855c06119c5fcdee75596,"All-New, All-Different Avengers (2015 - 2016)",2015,2015-10-07,Kenneth Rocafort,"Gerry Duggan, Mark Waid, James Robinson, G. Wi...",None,None,Comic,None,5.99,589


In [948]:
#PROYECCIÓN DE CAMPOS
projection_comics =pd.DataFrame(coleccion.find({}, {"comic_name": 1, "Price": 1, "_id": 0}))
print('Dataframe de comis con solo las columnas nombre y precio:')
projection_comics.head()

Dataframe de comis con solo las columnas nombre y precio:


,comic_name,Price
0,A Year of Marvels: April Infinite Comic (2016),0.0
1,A Year of Marvels: August Infinite Comic (2016),0.0
2,A Year of Marvels: February Infinite Comic (2016),0.0
3,A Year of Marvels: July Infinite Comic (2016),0.0
4,A Year of Marvels: June Infinite Comic (2016),0.0


In [949]:
#ORDENACION Y LÍMITE DEL CAMPO PRECIO LIMITADO A 3
df_top_priced = pd.DataFrame(coleccion.find().sort("Price", -1).limit(3))
print('Dataframe de top 3 comics más caros:')
df_top_priced.head()

Dataframe de top 3 comics más caros:


,_id,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id
0,699855c06119c5fcdee77fc1,Howard the Duck Annual (1977),1977,2008-07-23,None,None,None,MARVEL UNIVERSE,Hardcover,T+,99.99,11384
1,699855c06119c5fcdee77f7b,Howard the Duck (1976 - 1979),1976,2008-07-23,None,None,None,MARVEL UNIVERSE,Hardcover,T+,99.99,11314
2,699855c06119c5fcdee76cab,Daring Mystery Comics (1940 - 1942),1940,2008-01-23,None,None,None,MARVEL UNIVERSE,Hardcover,A,59.99,6498


### UPDATE

In [950]:
coleccion.update_one(
    {"id": 1},         
    {"$set": {"Rating": "All Ages"}} 
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [951]:
coleccion.update_many(
    {},  
    [{"$set": {"Rating": {"$toUpper": "$Rating"}}}] 
)

UpdateResult({'n': 34222, 'nModified': 25881, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### DELETE

In [952]:
coleccion.delete_one({"id": 1})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [953]:
coleccion.delete_many({"Price": 0})

DeleteResult({'n': 14470, 'ok': 1.0}, acknowledged=True)

## FASE 4 CURATED (limpieza y transformación)

In [954]:
df_cur = pd.DataFrame(list(db['raw_marvel'].find()))

### GESTIÓN DE NULOS

In [955]:
print(df_cur.isna().sum()/len(df_cur))

_id             0.000000
comic_name      0.000000
active_years    0.000000
publish_date    0.000000
penciler        0.259177
writer          0.192142
cover_artist    0.579363
Imprint         0.566098
Format          0.103792
Rating          0.000000
Price           0.103792
id              0.000000
dtype: float64


In [956]:
#IMPUTAMOS LOS NULOS DE PRECIO A LA MEDIANA
df_cur['Price']= df_cur['Price'].fillna(df_cur['Price'].median())

In [957]:
#IMPUTAMOS EL RATING A NO RATED
df_cur['Rating'] = df_cur['Rating'].fillna('Not Rated')
df_cur.head()

,_id,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id
0,699855c06119c5fcdee75354,A Year of Marvels: The Amazing (2016),2016,2016-04-27,"Danilo S. Beyruth, Ryan Browne","Ryan North, Amy Chu",Jamal Campbell,None,Comic,,4.99,11
1,699855c06119c5fcdee75355,A Year of Marvels: The Incredible (2016),2016,2016-06-15,Leonardo Romero,Dennis Culver,Jamal Campbell,None,Comic,,4.99,12
2,699855c06119c5fcdee75356,A Year of Marvels: The Uncanny (2016),2016,2016-12-07,None,None,None,Marvel Universe,Comic,T+,4.99,13
3,699855c06119c5fcdee75357,A+X (2012 - 2014),2012,2014-03-26,"Matteo Lolli, David Yardin, Will Sliney","Gerry Duggan, Jim Krueger",Kevin Nowlan,None,Comic,,3.99,14
4,699855c06119c5fcdee75358,A+X (2012 - 2014),2012,2014-02-19,"Paco Diaz, David Yardin","Gerry Duggan, Jeff Loveness",Goran Parlov,None,Comic,T,3.99,15


In [958]:
#IMPUTAMOS LOS NULOS DE IMPRENTA Y FORMATO
#LOS DOS SE IMPUTAN AL VALOR MAS FRECUENTE AL SER POCOS NULOS

cols_to_fix = ['Imprint', 'Format']
for col in cols_to_fix:
    df_cur[col] = df_cur[col].astype(str).str.strip().str.upper()
    df_cur[col] = df_cur[col].replace(['NAN', 'NONE', 'NULL', ''], np.nan)

imputer = SimpleImputer(strategy='most_frequent')

df_cur[cols_to_fix] = imputer.fit_transform(df_cur[cols_to_fix])


In [959]:
#IMPUTAMOS LOS TIPOS DE ARTISTAS EN EL VALOR UNKNOWN
author_cols = ['penciler', 'writer', 'cover_artist']

for col in author_cols:
    df_cur[col] = df_cur[col].astype(str).str.strip().replace(['nan', 'None', 'NULL', ''], np.nan)

from sklearn.impute import SimpleImputer
imputer_author = SimpleImputer(strategy='constant', fill_value='Unknown')

df_cur[author_cols] = imputer_author.fit_transform(df_cur[author_cols])


In [ ]:

print(df_cur.isna().sum()/len(df_cur))

_id             0.0
comic_name      0.0
active_years    0.0
publish_date    0.0
penciler        0.0
writer          0.0
cover_artist    0.0
Imprint         0.0
Format          0.0
Rating          0.0
Price           0.0
id              0.0
dtype: float64


### CORRECCIÓN DE TIPOS

In [961]:
cat_cols=['Rating', 'Format','Imprint']
df_cur[cat_cols]=df_cur[cat_cols].astype('category')
df_cur.dtypes

_id                     object
comic_name              object
active_years             int64
publish_date    datetime64[ns]
penciler                object
writer                  object
cover_artist            object
Imprint               category
Format                category
Rating                category
Price                  float64
id                       int64
dtype: object

## FASE 5 ANALYTICS (agregaciones)

## FASE 6 Rendimiento y diseño

## FASE 7 Visualización y conclusiones 